# data loading

In [1]:
pip install magenta

Note: you may need to restart the kernel to use updated packages.Collecting magenta
  Using cached magenta-2.1.4-py3-none-any.whl (1.4 MB)
  Using cached absl_py-1.2.0-py3-none-any.whl (123 kB)
  Using cached dm_sonnet-2.0.0-py3-none-any.whl (254 kB)
  Using cached imageio-2.20.0-py3-none-any.whl (3.4 MB)
  Using cached librosa-0.7.2-py3-none-any.whl
  Using cached matplotlib-3.5.2.tar.gz (35.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): still running...
  Preparing metadata (setup.py): finished with status 'done'
  Using cached mido-1.2.6-py2.py3-none-any.whl (69 kB)
  Using cached mir_eval-0.7-py3-none-any.whl
  Using cached note_seq-0.0.3-py3-none-any.whl (210 kB)
  Using cached numba-0.49.1.tar.gz (2.0 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of magenta to determine which version is compatible with other requirements. This could take a while.


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [847 lines of output]
  C:\Users\saiya\AppData\Local\Temp\pip-install-uzk515u7\numba_009090c184dc442fa43f51846c01ea80\setup.py:94: DeprecationWarning:
  
    `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
    of the deprecation of `distutils` itself. It will be removed for
    Python >= 3.12. For older Python versions it will remain present.
    It is recommended to use `setuptools < 60.0` for those Python versions.
    For more details, see:
      https://numpy.org/devdocs/reference/distutils_status_migration.html
  
  
    import numpy.distutils.misc_util as np_misc
  TBB not found
  OpenMP disabled
  C:\Users\saiya\anaconda3\Lib\site-packages\setuptools\__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
  !!
  
          ********************************************************************************
      

# data preprocessing


In [6]:
import os
from magenta.music.protobuf import music_pb2
from magenta.music import midi_io

# Function to process MIDI files and create NoteSequences
def create_note_sequences_from_midi(directory):
    note_sequences = []
    
    # Iterate through MIDI files in the directory
    for midi_file in os.listdir(directory):
        if midi_file.endswith(".mid"):
            file_path = os.path.join(directory, midi_file)
            
            # Read MIDI file and convert to NoteSequence
            midi_data = midi_io.midi_file_to_note_sequence(file_path)
            
            # Append the NoteSequence to the list
            note_sequences.append(midi_data)
    
    return note_sequences

# Directory containing MIDI files
midi_directory = "C:/Users/saiya/OneDrive/Desktop/mid-type"

# Create NoteSequences from MIDI files in the directory
sequences = create_note_sequences_from_midi(midi_directory)

# Print the number of NoteSequences created
print(f"Number of NoteSequences created: {len(sequences)}")


ModuleNotFoundError: No module named 'magenta'

In [7]:
import csv

# Function to convert NoteSequences to CSV
def save_sequences_to_csv(sequences, csv_filename):
    with open(csv_filename, mode='w', newline='') as csv_file:
        fieldnames = ['note_number', 'note_start_time', 'note_end_time', 'velocity', 'instrument']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        writer.writeheader()  # Write CSV header
        
        for sequence in sequences:
            for note in sequence.notes:
                writer.writerow({
                    'note_number': note.pitch,
                    'note_start_time': note.start_time,
                    'note_end_time': note.end_time,
                    'velocity': note.velocity,
                    'instrument': note.instrument,
                })

# Name of the CSV file to save
output_csv_filename = 'output_notes.csv'

# Save NoteSequences to CSV
save_sequences_to_csv(sequences, output_csv_filename)

print(f"NoteSequences saved to {output_csv_filename}")

NameError: name 'sequences' is not defined

# data exploration and  refining

# initializing Generator and discriminator


In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, Dropout, Input

# Function to build the generator using LSTM
from keras.models import Sequential, Model
from keras.layers import LSTM, Dropout, Dense, Input, BatchNormalization, Reshape

def build_generator(latent_dim, sequence_length, num_features):
    generator = Sequential()

    # Existing layers
    generator.add(LSTM(256, input_shape=(latent_dim,), return_sequences=True))
    generator.add(Dropout(0.3))
    generator.add(LSTM(128))
    generator.add(Dense(sequence_length * num_features, activation='tanh'))
    generator.add(BatchNormalization())
    generator.add(Dense(sequence_length * num_features, activation='sigmoid'))
    generator.add(Reshape((sequence_length, num_features)))
    
    # Adding 10 more LSTM layers with dropout
    for _ in range(10):
        generator.add(LSTM(64, return_sequences=True))
        generator.add(Dropout(0.3))

    gen_input = Input(shape=(latent_dim,))
    gen_output = generator(gen_input)
    
    return Model(gen_input, gen_output)


# Function to build the discriminator using LSTM
from keras.models import Sequential, Model
from keras.layers import LSTM, Dropout, Dense, Input

def build_discriminator(sequence_length, num_features):
    discriminator = Sequential()
    
    # Adding 10 LSTM layers with dropout
    discriminator.add(LSTM(256, input_shape=(sequence_length, num_features), return_sequences=True))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(LSTM(128, return_sequences=True))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(LSTM(64, return_sequences=True))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(LSTM(32, return_sequences=True))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(LSTM(16, return_sequences=True))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(LSTM(8, return_sequences=True))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(LSTM(4, return_sequences=True))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(LSTM(2, return_sequences=True))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(LSTM(1))  # Last LSTM layer without return_sequences
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(1, activation='sigmoid'))
    
    disc_input = Input(shape=(sequence_length, num_features))
    disc_output = discriminator(disc_input)
    
    return Model(disc_input, disc_output)

# Set dimensions
latent_dim = 100  # Latent dimension for the generator
sequence_length = 50  # Length of the music sequence
num_features = 128  # Number of features in the sequence

# Build generator and discriminator
generator = build_generator(latent_dim, sequence_length, num_features)
discriminator = build_discriminator(sequence_length, num_features)

# Compile the discriminator
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

# Combined GAN model
discriminator.trainable = False
gan_input = Input(shape=(latent_dim,))
generated_sequence = generator(gan_input)
gan_output = discriminator(generated_sequence)

gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')


# initializing magenta's melody RNN model


In [1]:
import os
import magenta.music as mm
from magenta.models.music_vae import TrainedModel
from magenta.models.music_vae import configs

# Define a directory to save model checkpoints
checkpoint_dir = 'path_to_save_checkpoints'  # Set your desired checkpoint directory

# Define the MelodyRNN configuration
melody_rnn_config = configs.CONFIG_MAP['melody_rnn']  # You can choose other configs as well

# Initialize the MelodyRNN model
melody_rnn = TrainedModel(
    config=melody_rnn_config,
    batch_size=4,
    checkpoint_dir_or_path=checkpoint_dir
)

# Prepare the dataset for training
# (Assuming 'loaded_sequences' is a list of NoteSequences loaded from your CSV)
note_sequences = loaded_sequences[:100]  # Use a subset of sequences for training

# Train the MelodyRNN model
melody_rnn.train(note_sequences, num_steps=1000)

# Generate melodies using the trained model
generated_sequences = melody_rnn.generate()


# GAN training loop using melody RNN in generator

In [ ]:
from magenta.models.melody_rnn import melody_rnn_sequence_generator

# Define the directory where the MelodyRNN model is saved
melody_rnn_checkpoint_dir = 'path_to_your_model_checkpoint_directory'

# Initialize the MelodyRNN model from the saved checkpoint directory
melody_rnn = melody_rnn_sequence_generator.MelodyRnnSequenceGenerator(
    model=None,  # Pass None as model name when loading from checkpoint
    checkpoint_dir=melody_rnn_checkpoint_dir
)

# Function to generate sequences using Magenta MelodyRNN model
def generate_magenta_sequences(num_sequences):
    generated_sequences = []
    for _ in range(num_sequences):
        generated_sequence = melody_rnn.generate()  # Generate a sequence using the MelodyRNN model
        generated_sequences.append(generated_sequence)
    return generated_sequences

# Training loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    
    # Iterate through batches of NoteSequences data
    for i in range(0, len(X_train), batch_size):
        real_sequences = X_train[i:i+batch_size]  # Get a batch of real sequences
        
        # Generate fake sequences using Magenta MelodyRNN model
        generated_sequences = generate_magenta_sequences(batch_size)
        
        # Train the discriminator and generator as before...
        # (Your existing code for training the discriminator and generator goes here)

    # Print the losses for each epoch
    print(f"Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")


# Generating music sequences using melody rnn generator

In [ ]:
# Assuming you have trained GAN and MelodyRNN models
# Define your trained models (gan, melody_rnn)

# 1. Generate Music using the GAN
def generate_music_with_gan():
    # Generate latent vectors using GAN
    latent_vectors = generate_latent_vectors()  # Implement your function to generate latent vectors
    
    # Generate sequences using GAN's generator
    generated_sequences = gan_generator.predict(latent_vectors)  # Use your GAN's generator to create sequences
    
    return generated_sequences

# 2. Generate Music using MelodyRNN
def generate_music_with_melody_rnn():
    # Generate sequences using MelodyRNN model
    generated_sequences = melody_rnn.generate()  # Use your trained MelodyRNN model to generate sequences
    
    return generated_sequences

# 3. Combine GAN and MelodyRNN generated sequences
def combine_sequences(gan_sequences, melody_rnn_sequences):
    # Manipulate or combine the sequences as needed
    combined_sequences = manipulate_sequences(gan_sequences, melody_rnn_sequences)  # Implement your combining function
    
    return combined_sequences

# Generate music using GAN
gan_sequences = generate_music_with_gan()

# Generate music using MelodyRNN
melody_rnn_sequences = generate_music_with_melody_rnn()

# Combine sequences from GAN and MelodyRNN
combined_sequences = combine_sequences(gan_sequences, melody_rnn_sequences)

# Save or use the combined sequences for music playback or further processing
save_sequences_to_midi(combined_sequences)  # Save the sequences to MIDI or use them for playback


In [ ]:
import mido

def save_sequences_to_midi(sequences, output_file):
    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)

    for sequence in sequences:
        for note in sequence:
            track.append(mido.Message('note_on', note=note.pitch, velocity=note.velocity, time=int(note.start_time * 1000)))
            track.append(mido.Message('note_off', note=note.pitch, velocity=note.velocity, time=int(note.end_time * 1000)))

    mid.save(output_file)

# Usage
save_sequences_to_midi(combined_sequences, 'generated_music.mid')
